# Gán nhãn từ loại với mô hình Markov ẩn
Notebook này hướng dẫn sử dụng mô hình Markov ẩn (HMM) để thực hiện gán nhãn dữ liệu. Chúng ta sẽ sử dụng Python để xây dựng một mô hình gán nhãn dữ liệu bằng HMM và thuật toán Veterbi

Đọc dữ liệu và tổ chức dữ liệu huấn luyện

In [1]:
# Importing libraries
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time
 
#download the treebank corpus from nltk
nltk.download('treebank')
 
#download the universal tagset from nltk
nltk.download('universal_tagset')
 
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))
 
#print the first two sentences along with tags
print(nltk_data[:2])

[nltk_data] Downloading package treebank to /home/giabao/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /home/giabao/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')]]


In [2]:
#print each word with its respective tag for first two sentences
for sent in nltk_data[:2]:
  for tuple in sent:
    print(tuple)

('Pierre', 'NOUN')
('Vinken', 'NOUN')
(',', '.')
('61', 'NUM')
('years', 'NOUN')
('old', 'ADJ')
(',', '.')
('will', 'VERB')
('join', 'VERB')
('the', 'DET')
('board', 'NOUN')
('as', 'ADP')
('a', 'DET')
('nonexecutive', 'ADJ')
('director', 'NOUN')
('Nov.', 'NOUN')
('29', 'NUM')
('.', '.')
('Mr.', 'NOUN')
('Vinken', 'NOUN')
('is', 'VERB')
('chairman', 'NOUN')
('of', 'ADP')
('Elsevier', 'NOUN')
('N.V.', 'NOUN')
(',', '.')
('the', 'DET')
('Dutch', 'NOUN')
('publishing', 'VERB')
('group', 'NOUN')
('.', '.')


In [3]:
# split data into training and validation set in the ratio 80:20
# YOUR CODE HERE

train_set, test_set = train_test_split(nltk_data, test_size=0.2, random_state=42)   



In [4]:
# create list of train and test tagged words
# YOUR CODE HERE
train_tagged_words = [tup for sent in train_set for tup in sent]    
test_tagged_words = [tup for sent in test_set for tup in sent]

print(len(train_tagged_words))
print(len(test_tagged_words))

80127
20549


In [5]:
# check some of the tagged words.
train_tagged_words[:5]

[('Pierre', 'NOUN'),
 ('Vinken', 'NOUN'),
 (',', '.'),
 ('61', 'NUM'),
 ('years', 'NOUN')]

In [6]:
#use set datatype to check how many unique tags are present in training data
tags = {tag for word,tag in train_tagged_words}
print(len(tags))
print(tags)
 
# check total words in vocabulary
vocab = {word for word,tag in train_tagged_words}

12
{'CONJ', 'ADP', 'X', 'ADV', 'VERB', 'NUM', '.', 'DET', 'ADJ', 'PRT', 'NOUN', 'PRON'}


Tính Emission Probability và Transition Probability từ dữ liệu huấn luyện

In [7]:
# compute Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    # YOUR CODE HERE
#now calculate the total number of times the passed word occurred as the passed tag.
    # YOUR CODE HERE
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)

 
    return (count_w_given_tag, count_tag)

In [8]:
# compute  Transition Probability
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [9]:
# creating t x t transition matrix of tags, t= no of tags
# Matrix(i, j) represents P(jth tag after the ith tag)
 
tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]
 
print(tags_matrix)

[[5.57413616e-04 5.35117052e-02 7.80379027e-03 5.68561889e-02
  1.51059091e-01 3.95763665e-02 2.95429211e-02 1.21516168e-01
  1.20958753e-01 3.90189514e-03 3.53400230e-01 6.13154955e-02]
 [1.02184189e-03 1.64771993e-02 3.24434787e-02 1.37948655e-02
  8.68565589e-03 6.30987361e-02 3.92131805e-02 3.27628046e-01
  1.03972413e-01 1.27730239e-03 3.23412955e-01 6.89743236e-02]
 [1.10413097e-02 1.44869596e-01 7.55758584e-02 2.68418044e-02
  2.02741295e-01 3.04587861e-03 1.61431566e-01 5.40643446e-02
  1.52293928e-02 1.86179325e-01 6.18694089e-02 5.71102239e-02]
 [7.48916063e-03 1.22585729e-01 2.44383123e-02 8.43515992e-02
  3.38983059e-01 3.11391409e-02 1.35199055e-01 6.54316097e-02
  1.30863219e-01 1.34016555e-02 3.07449736e-02 1.53724868e-02]
 [5.08130062e-03 9.13710296e-02 2.19235033e-01 8.11160356e-02
  1.71008870e-01 2.34663710e-02 3.47376205e-02 1.32390976e-01
  6.47634864e-02 3.02106421e-02 1.09848484e-01 3.67701389e-02]
 [1.38346935e-02 3.54735740e-02 2.01489896e-01 3.54735716e-03
  1

In [10]:
# convert the matrix to a df for better readability
#the table is same as the transition table shown in section 3 of article
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))
display(tags_df)

,CONJ,ADP,X,ADV,VERB,NUM,.,DET,ADJ,PRT,NOUN,PRON
CONJ,0.000557,0.053512,0.007804,0.056856,0.151059,0.039576,0.029543,0.121516,0.120959,0.003902,0.353400,0.061315
ADP,0.001022,0.016477,0.032443,0.013795,0.008686,0.063099,0.039213,0.327628,0.103972,0.001277,0.323413,0.068974
X,0.011041,0.144870,0.075576,0.026842,0.202741,0.003046,0.161432,0.054064,0.015229,0.186179,0.061869,0.057110
ADV,0.007489,0.122586,0.024438,0.084352,0.338983,0.031139,0.135199,0.065432,0.130863,0.013402,0.030745,0.015372
VERB,0.005081,0.091371,0.219235,0.081116,0.171009,0.023466,0.034738,0.132391,0.064763,0.030211,0.109848,0.036770
NUM,0.013835,0.035474,0.201490,0.003547,0.017027,0.184108,0.119191,0.003193,0.031571,0.029443,0.359347,0.001774
.,0.057259,0.089948,0.027454,0.052452,0.090589,0.079906,0.095182,0.171670,0.044333,0.002457,0.222626,0.066019
DET,0.000433,0.009235,0.045022,0.011833,0.038672,0.022655,0.018326,0.005339,0.207359,0.000289,0.637229,0.003608
ADJ,0.017910,0.077741,0.021059,0.004723,0.012793,0.018303,0.065341,0.005117,0.066916,0.009841,0.699469,0.000787
PRT,0.002343,0.022257,0.013667,0.010543,0.404920,0.059352,0.042171,0.100351,0.088637,0.001562,0.237407,0.016790


Xây dựng thuật toán Viterbi

In [11]:
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

Test thuật toán Viterbi

In [12]:
# Let's test our Viterbi algorithm on a few sample sentences of test dataset
random.seed(1234)      #define a random seed to get same sentences when run multiple times
 
# choose random 10 numbers
rndom = [random.randint(1,len(test_set)) for x in range(10)]
 
# list of 10 sents on which we test the model
test_run = [test_set[i] for i in rndom]
 
# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]
 
# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]

In [13]:
#Here We will only test 10 sentences to check the accuracy
#as testing the whole training set takes huge amount of time
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
 
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  25.746501207351685
Viterbi Algorithm Accuracy:  88.96321070234113


In [14]:
#Code to test all the test sentences
#(takes alot of time to run)
# tagging the test sentences()
# YOUR CODE HERE

test_untagged_words = [tup[0] for sent in test_set for tup in sent]
 
start = time.time()
tagged_seq = Viterbi(test_untagged_words)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
 
# accuracy
check = [i for i, j in zip(test_tagged_words, test_untagged_words) if i == j] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

KeyboardInterrupt: 

Tăng độ chính xác bằng cách kết hợp thêm rule

In [15]:
#To improve the performance,we specify a rule base tagger for unknown words 
# specify patterns for tagging
patterns = [
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*ed$', 'VERB'),               # past tense 
    (r'.*es$', 'VERB'),               # verb    
    (r'.*\'s$', 'NOUN'),              # possessive nouns
    (r'.*s$', 'NOUN'),                # plural nouns
    (r'\*T?\*?-[0-9]+$', 'X'),        # X
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
    (r'.*', 'NOUN')                   # nouns
]
 
# rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)

In [16]:
#modified Viterbi to include rule based tagger in it
def Viterbi_rule_based(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        state_max = rule_based_tagger.tag([word])[0][1]       
        
         
        if(pmax==0):
            state_max = rule_based_tagger.tag([word])[0][1] # assign based on rule based tagger
        else:
            if state_max != 'X':
                # getting state for which probability is maximum
                state_max = T[p.index(pmax)]                
             
         
        state.append(state_max)
    return list(zip(words, state))

In [17]:
#test accuracy on subset of test data 
start = time.time()
tagged_seq = Viterbi_rule_based(test_tagged_words)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
 
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  22.01218318939209
Viterbi Algorithm Accuracy:  95.31772575250837


In [18]:
#Check how a sentence is tagged by the two POS taggers
#and compare them
test_sent="Will can see Marry"
pred_tags_rule=Viterbi_rule_based(test_sent.split())
pred_tags_withoutRules= Viterbi(test_sent.split())
print(pred_tags_rule)
print(pred_tags_withoutRules)
#Will and Marry are tagged as NUM as they are unknown words for Viterbi Algorithm 

[('Will', 'NOUN'), ('can', 'VERB'), ('see', 'VERB'), ('Marry', 'NOUN')]
[('Will', 'CONJ'), ('can', 'VERB'), ('see', 'VERB'), ('Marry', 'CONJ')]
